<a href="https://colab.research.google.com/github/ibukikuroyanagi/Assignment/blob/master/make_sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
user_id = "M04Kaz"
CONFIG = {
"CONSUMER_KEY": 'uG33PVdKk3CXQCDj8Y60ZGZ1k',
"CONSUMER_SECRET" : 'YQKaksOWtFOWkj2txEXkA7mWvqhxYJzzchrb6dBDe9ZtCUWoWN',
"ACCESS_TOKEN" : '1054515250639196162-k21qwKXpn7HdvwbVUZTtgiDsIazgkV',
"ACCESS_SECRET" : 'PIhmOqIJTmue4QPTDGhr2LoDTvGVpzlagk6t8PJC77JCR',
}

In [0]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

# 認証処理
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())


In [6]:
import tweepy
import pandas as pd
from time import time

#1 各種ツイッターのキーをセット

CONSUMER_KEY = CONFIG["CONSUMER_KEY"]
CONSUMER_SECRET = CONFIG["CONSUMER_SECRET"]
ACCESS_TOKEN = CONFIG["ACCESS_TOKEN"]
ACCESS_SECRET = CONFIG["ACCESS_SECRET"]

#2 Tweepy
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

#3 APIインスタンスを作成
api = tweepy.API(auth)
user = api.get_user(screen_name=user_id)

def get_profile(user_id):
    user = api.get_user(screen_name=user_id)
    profile = {
        "id":user.id,
        "user_id":user_id,
        "name":user.name,
        "description":user.description,
        "statuses_count":user.statuses_count,
        "followers_count":user.followers_count
    }
    return profile

user_data = get_profile(user_id=user_id)

# 変数データを作成
name = user_data["name"]
statuses_count = 990
followers_count = 9999

if statuses_count > user_data["statuses_count"]:
  statuses_count = user_data["statuses_count"]
if followers_count > user_data["followers_count"]:
  followers_count = user_data["followers_count"]

screen_name = user_id

# -*- coding: utf-8 -*-

from requests_oauthlib import OAuth1Session
import json
import datetime, time, sys
from abc import ABCMeta, abstractmethod


class TweetsGetter(object):
    __metaclass__ = ABCMeta

    def __init__(self):
        self.session = OAuth1Session(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_SECRET)

    @abstractmethod
    def specifyUrlAndParams(self, keyword):
        '''
        呼出し先 URL、パラメータを返す
        '''

    @abstractmethod
    def pickupTweet(self, res_text, includeRetweet):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''

    @abstractmethod
    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''

    def collect(self, total = -1, onlyText = False, includeRetweet = False):
        '''
        ツイート取得を開始する
        '''

        #----------------
        # 回数制限を確認
        #----------------
        self.checkLimit()

        #----------------
        # URL、パラメータ
        #----------------
        url, params = self.specifyUrlAndParams()
        params['include_rts'] = str(includeRetweet).lower()
        # include_rts は statuses/user_timeline のパラメータ。search/tweets には無効

        #----------------
        # ツイート取得
        #----------------
        cnt = 0
        unavailableCnt = 0
        while True:
            res = self.session.get(url, params = params)
            if res.status_code == 503:
                # 503 : Service Unavailable
                if unavailableCnt > 10:
                    raise Exception('Twitter API error %d' % res.status_code)

                unavailableCnt += 1
                print ('Service Unavailable 503')
                self.waitUntilReset(time.mktime(datetime.datetime.now().timetuple()) + 30)
                continue

            unavailableCnt = 0

            if res.status_code != 200:
                raise Exception('Twitter API error %d' % res.status_code)

            tweets = self.pickupTweet(json.loads(res.text))
            if len(tweets) == 0:
                # len(tweets) != params['count'] としたいが
                # count は最大値らしいので判定に使えない。
                # ⇒  "== 0" にする
                # https://dev.twitter.com/discussions/7513
                break

            for tweet in tweets:
                if (('retweeted_status' in tweet) and (includeRetweet is False)):
                    pass
                else:
                    if onlyText is True:
                        yield tweet['text']
                    else:
                        yield tweet

                    cnt += 1
                    if cnt % 100 == 0:
                        print ('%d件 ' % cnt)

                    if total > 0 and cnt >= total:
                        return

            params['max_id'] = tweet['id'] - 1

            # ヘッダ確認 （回数制限）
            # X-Rate-Limit-Remaining が入ってないことが稀にあるのでチェック
            if ('X-Rate-Limit-Remaining' in res.headers and 'X-Rate-Limit-Reset' in res.headers):
                if (int(res.headers['X-Rate-Limit-Remaining']) == 0):
                    self.waitUntilReset(int(res.headers['X-Rate-Limit-Reset']))
                    self.checkLimit()
            else:
                print ('not found  -  X-Rate-Limit-Remaining or X-Rate-Limit-Reset')
                self.checkLimit()

    def checkLimit(self):
        '''
        回数制限を問合せ、アクセス可能になるまで wait する
        '''
        unavailableCnt = 0
        while True:
            url = "https://api.twitter.com/1.1/application/rate_limit_status.json"
            res = self.session.get(url)

            if res.status_code == 503:
                # 503 : Service Unavailable
                if unavailableCnt > 10:
                    raise Exception('Twitter API error %d' % res.status_code)

                unavailableCnt += 1
                print ('Service Unavailable 503')
                self.waitUntilReset(time.mktime(datetime.datetime.now().timetuple()) + 30)
                continue

            unavailableCnt = 0

            if res.status_code != 200:
                raise Exception('Twitter API error %d' % res.status_code)

            remaining, reset = self.getLimitContext(json.loads(res.text))
            if (remaining == 0):
                self.waitUntilReset(reset)
            else:
                break

    def waitUntilReset(self, reset):
        '''
        reset 時刻まで sleep
        '''
        seconds = reset - time.mktime(datetime.datetime.now().timetuple())
        seconds = max(seconds, 0)
        print ('\n     =====================')
        print ('     == waiting %d sec ==' % seconds)
        print ('     =====================')
        sys.stdout.flush()
        time.sleep(seconds + 10)  # 念のため + 10 秒

    @staticmethod
    def bySearch(keyword):
        return TweetsGetterBySearch(keyword)

    @staticmethod
    def byUser(screen_name):
        return TweetsGetterByUser(screen_name)


class TweetsGetterBySearch(TweetsGetter):
    '''
    キーワードでツイートを検索
    '''
    def __init__(self, keyword):
        super(TweetsGetterBySearch, self).__init__()
        self.keyword = keyword
        
    def specifyUrlAndParams(self):
        '''
        呼出し先 URL、パラメータを返す
        '''
        url = 'https://api.twitter.com/1.1/search/tweets.json'
        params = {'q':self.keyword, 'count':100}
        return url, params

    def pickupTweet(self, res_text):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''
        results = []
        for tweet in res_text['statuses']:
            results.append(tweet)

        return results

    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''
        remaining = res_text['resources']['search']['/search/tweets']['remaining']
        reset     = res_text['resources']['search']['/search/tweets']['reset']

        return int(remaining), int(reset)
    

class TweetsGetterByUser(TweetsGetter):
    '''
    ユーザーを指定してツイートを取得
    '''
    def __init__(self, screen_name):
        super(TweetsGetterByUser, self).__init__()
        self.screen_name = screen_name
        
    def specifyUrlAndParams(self):
        '''
        呼出し先 URL、パラメータを返す
        '''
        url = 'https://api.twitter.com/1.1/statuses/user_timeline.json'
        params = {'screen_name':self.screen_name, 'count':200}
        return url, params

    def pickupTweet(self, res_text):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''
        results = []
        for tweet in res_text:
            results.append(tweet)

        return results

    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''
        remaining = res_text['resources']['statuses']['/statuses/user_timeline']['remaining']
        reset     = res_text['resources']['statuses']['/statuses/user_timeline']['reset']

        return int(remaining), int(reset)

# 数字からアルファベットを返す関数
# 例：26→Z、27→AA、10000→NTP
def toAlpha(num):
    if num<=26:
        return chr(64+num)
    elif num%26==0:
        return toAlpha(num//26-1)+chr(90)
    else:
        return toAlpha(num//26)+chr(64+num%26)
      
# dfをworksheetに代入する関数
def substitute_data_for_sheet(df,worksheet):
    col_lastnum = len(df.columns) # DataFrameの列数
    row_lastnum = len(df.index)   # DataFrameの行数

    cell_list = worksheet.range('A1:'+toAlpha(col_lastnum+1)+str(row_lastnum+1))
    for cell in cell_list:
        if cell.col == 1 and cell.row == 1:
            val = ''
        elif cell.col == 1:
            val = df.index[cell.row-2]
        elif cell.row == 1:
            val = df.columns[cell.col-2]
        else:
            val = df.iloc[cell.row-2][cell.col-2]
        cell.value = val
    worksheet.update_cells(cell_list)

# キーワードで取得
#getter = TweetsGetter.bySearch(u'渋谷')
    
# ユーザーを指定して取得 （screen_name）
getter = TweetsGetter.byUser(screen_name)


columns = [ "screen_name","id","created_at", "text","retweet_count","favorite_count"]
df = pd.DataFrame(columns=columns)

for tweet in getter.collect():
    print(tweet)
    _id = tweet["id"]
    created_at = tweet['created_at']
    text = tweet['text']
    retweet_count = tweet['retweet_count']
    favorite_count = tweet['favorite_count']
    se = pd.Series([screen_name,_id,created_at, text,retweet_count,favorite_count],columns)
    df = df.append(se,ignore_index=True)
    
# df["created_at"] = pd.to_datetime(df["created_at"])
print(df)
#スプレッドシートを新規作成
workbook = gc.create(name)
worksheet = workbook.get_worksheet(0)#先頭のシートを指定
worksheet.update_title("tweets_count")	# ワークシート名を変更する
worksheet = workbook.get_worksheet(0)
substitute_data_for_sheet(dff,worksheet)# ツイートをシートに代入
# ワークシートを末尾に新規作成する
# [補足]100行26列の新しいワークシートを作成
workbook.add_worksheet(title="profile",rows=followers_count+50,cols=26)

# Pandasで行の作成
columns = ["screen_name", "description", "follower", "following"]
df2 = pd.DataFrame(columns=columns)
target_screen_name = screen_name
followers_ids = tweepy.Cursor(api.followers_ids, id = target_screen_name, cursor = -1).items(followers_count)
followers_ids_list = []
try:
    for followers_id in followers_ids:
        followers_ids_list.append(followers_id)
except tweepy.error.TweepError as e:
    print(e.reason)


for follower_id in followers_ids_list[:followers_count]:
    try:
        user = api.get_user(follower_id)
        screen_name = user.screen_name
        description = user.description
        follower = user.followers_count
        following = user.friends_count
        se = pd.Series([screen_name, description, follower, following], columns)
        df2 = df2.append(se, ignore_index=True)
        print(df2)
    except Exception as e:
        print(e)
        time.sleep(60*15)


worksheet = workbook.worksheet("profile")
# プロフィールをシートに代入
substitute_data_for_sheet(df2,worksheet)


{'created_at': 'Mon May 06 14:24:15 +0000 2019', 'id': 1125405922644221952, 'id_str': '1125405922644221952', 'text': 'progateをマスターしてきたので、新しくTechpitを利用して学習していく！\n\n#progate\n#techpit\n#駆け出しエンジニア https://t.co/xNG9C3xzfc', 'truncated': False, 'entities': {'hashtags': [{'text': 'progate', 'indices': [43, 51]}, {'text': 'techpit', 'indices': [52, 60]}, {'text': '駆け出しエンジニア', 'indices': [61, 71]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/xNG9C3xzfc', 'expanded_url': 'https://twitter.com/kohe91/status/1050332927446249472', 'display_url': 'twitter.com/kohe91/status/…', 'indices': [72, 95]}]}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1117252412589240320, 'id_str': '1117252412589240320', 'name': 'kaz / エンジニア1年目', 'screen_name': 'M04Kaz', 'loca